# Picúr, az éhes hörcsög

Úgy érzitek, hogy most már elég idősek, érettek és felelősség tudatosak vagytok és eljött az ideje, hogy beszerezzetek egy kicsi, de annál cukibb szíriai hörcsögöt. Picúrnak fogjátok hívni. Ő egy nagyon aranyos és virgonc kis hörcsög aki nagyon szereti a répát, de a répa megszerzésében nektek kell segíteni. Nektek kell megmutatni az utat és irányítani Picúrt, hogy megtalálja a répát. 

## Mit fogsz készíteni?

Egy egyszerű számítógépes játékot fogunk készíteni, amiben egy Picúr nevű hörcsögöt fogunk három gomb segítségével irányítani jobbra, balra, le- és felfele. A célja a játéknak, hogy Picúr megtalálja a répát. 

## Mit tanulsz meg?

A játék elkészítésével a következőket tanulod meg:

* Hogyan tudsz több gombot egy áramkörbe beiktatni
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal
* Hogyan kell egy grafikus felületet létrehozni a **pygame** modullal
* Hogyan hangold össze a grafikus megjelenítő felületet a gombokkal és mozgass elemeket a grafikus felületen

## A projekt részletekre bontása

* A **pygame** installálása
* Defininálni a le, fel, jobbra és balra irányokat
* Beszerezni képeket a hörcsögről és a répáról
* Elhelyezni ezeket a játéktéren
* Megakadályozni, hogy a hörcsög lemehessen a képernyőről
* Frissíteni a képek helyzetét a mozgatások után
* Kiírni a képernyőre, hogy Yummy ha a hörcsög megtalálta a répát

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) 3 db nyomó gomb: [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) vagy [építhetsz]()

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

## A kapcsolási rajz

<img src="prog09_circuit.png" width=600 height=400 />

1) A nyomógombot lábait illesszük a breadbord mélyedéseibe. Ügyeljünk arra, hogy a nyomógomb egy (vízszintes) sorba kerülő lábai a gomb lenyomása nélkül is kapcsolatban legyenek egymással, míg a (függőlegesen) egy sorba kerülő lábai csak a lenyomás következtében lesznek összekötve. Mindezt egy műszerrel leellenprízhetjük. 

2) Ugyanezt ismételjük meg még kétszer, ahogy az ábra mutatja. A két szélső gombok szolgálnak majd a következő és az előző zene elindítására, míg a középső kettős funkciót lát majd el, elindítja és leállítja a zenét. 

3) Fogjunk 3 jumper kábelt (pl. kék, zöld és narancssárga) és kössük egyenként egyik végüket a gomb egyik lábának sorába. A kék kábel másik végét a Raspberry Pi **GPIO 02**-es (vagy **3**-as) jelöléssel ellátott tüskéjére, a zöld kábel másik végét a **GPIO 03**-es (vagy **5**-ös) tüskére és a narancssárga kábel másik végét a **GPIO 17**-es (vagy **11**-es) tüskére kössük. 

4) A gombok másik lábának sorába pedig földelést kell kötni. Fogjunk sárga jumper kábelt és a gombok szabadon lévő lábának sorába tegyük a kábel egyik végét, míg a másikat a szélső függőlegesen összekötött oszlopok egyikébe. Ugyanabba az oszlopba kössünk be egy jumper kábelt, aminek a másik vége a Raspberry Pi **GND** (pl. **34**-es) jelöléssel ellátott tüskéjéhez kapcsolódik. Ez a tüske alapértelmezetten földelés.

## A kód

Ha valaki játékokat szeretne fejleszteni Pythonbó, annak a [*pygame*](https://www.pygame.org/news) modul egy kézenfekvő csomag. Ez lehetővé teszi a felhasználónak, hogy könnyedén és rövid idő alatt multimédiás termékeket hozzon létre. A *pygame* általában alapjáraton fel van installálva a Raspbian operációs rendszeren de jól utána kell járni, hogy milyen Python verzióhoz tartozik.

Amennyiben a te számítógépeden nem lenne felinstallálva a *pygame* (amit a ```pip list``` paranccsal tudsz ellenőrizni), nyiss meg egy terminált és írd be hogy ```pip install pygame```. Ha nem kaptál hibaüzenetet, akkor ezután már rendelkezned kellene evvel a csomaggal. 

A kódban szükségünk lesz még egy hörcsög és egy répa képére is. Ezeket bárhonnan be lehet tölteni az internetrő, de mi a [http://www.freepik.com/](http://www.freepik.com/) weboldalról keresgéltünk.

A kódunk első részében, mint mindig, először beimportálunk néhány modult. A ```gpiozero``` modulból a ```Button``` objektumot ami lehetővé teszi, hogy a Raspi kommunikáljon a gombbal a **GPIO** tüskéken keresztül. Emellett betöltjük a **time** modult amit időmérésre fogunk majd használni, a **random** modult amit a véletlenszerű számok generálására használunk, a **sys** modult, ami lehetővé teszi, hogy pythonból elérjük az operációs rendszer parancsait, a **pygame** csomagot ami a grafikai megjelenítésért felel.

In [ ]:
from gpiozero import Button
import time, random
import sys, pygame

A modulok beolvasása után inicializálunk három ```Button``` objektumot, amiket ```left```-nek, ```right```-nak és ```up_down```-nak nevezünk el. Ezeket a *17*-es, *2*-es és a *3*-ss számú **GPIO** tüskékre csatlakoztatjuk.

In [ ]:
right = Button(2)
up_down = Button(3)
left = Button(17)

A következő lépésben elkezdjük definiálni a grafikus felületünket. A ```pygame.init()``` közli a pythonnal, hogy készek vagyunk használni a grafikus felületet és elkezdjük azt inicializálni. A ```pygame.font.init()``` ugyanezt csinálja, csak a betűtípusokat inicializálja. Ezután létrehozunk egy ```font``` objektumot az operációs rendszerünk betűtípus készletéből a ```pygame.font.SysFont('Comic Sans MS', 30)``` paranccsal. 

In [ ]:
pygame.init( )
pygame.font.init()
font = pygame.font.SysFont('Comic Sans MS', 30)

Ezután definiálunk néhány előre meghatározott értéket a grafikus felületünkkel (canvas) kapcsolatban, mint pl. a szélesség (width), magasság (height), a háttér színe RGB stílusban (white) és a hörcsög lépéseinek nagysága (step).

In [1]:
#Set canvas parameters
width, height = 500, 500
size = width
white = 255,255,255
step = 50

Ebben a függvényben először hivatkozunk a ```path``` globális változónkra majd kinyomtatjuk a képernyőre, hogy indítjuk a zenét. A ```call``` paranccsal kommunikálunk a terminál ablakunkkal és utasítjuk azt, hogy hajtsa végre a ```mocp -S``` parancsot, ami a zenelejátszó szervert elindítja (erről a terminál válaszban értesít is minket). A ```call``` argumentuma egy lista amiben megadjuk mind a parancsszavat és a hozzá tartozó argumentumokat is a lista különböző elemeiként. API-n keresztül indítjuk el a zenét a ```moc.find_and_play(path + '/*')``` paranccsal, ami megkeresi a mappát ahol a zene fájlok találhatók, majd elindítja az elsőt. Az API dokumentációjában a ```find_and_play``` metódus mellett még sok másik is található.

Ezután vegyük át a többi függvényt is nagyvonalakban:

In [1]:
def stop_playing():
    print('Stopping the song')
    call(['mocp','-s'])

def play_next_song():
    print('Playing the next song')
    call(['mocp','--next'])

def play_previous_song():
    print('Playing the previous song')
    call(['mocp','--previous'])

Ezek a függvények mind a terminálos parancsok alapján utasítják a zenelejátszót a feladat elvégzésére. Mindegyik utasítás az ```mocp```-vel kezdődik és a hozzátartozó argumentum változik a feladat függvényében. 

Eddig definiáltunk négy függvényt, de csak három gombunk van. Mint ahogy írtuk az elején az egyik gomb, kettős funkciót fog ellátni (mert pl. a mérnökök kifogytak a pénzből és nem tudtak még egy gombot venni :)). A zene elindítását és megállítását egy gombhoz rendeljük, mégpedig attól függően, hogy milyen hosszan lesz lenyomva. Ha a lenyomás 1 másodpercnél rövidebb, akkor elindítja, ha hosszabb akkor megállítja a zenét.

Ehhez definiáljuk a ```start```, ```stop``` és ```encoder``` függvényeket:

In [2]:
def start():
    global press
    press = time.time()

def end():
    global release
    release = time.time()
    delay = release - press
    func = encoder(delay)
    func()

def encoder(delay):
    if delay < 1:
        return start_playing
    else:
        return stop_playing

A ```start``` függvény rögzíti a gomb lenyomásának időpillanatát a ```press``` globális változóba. Az ```end``` függvény egyrészt rögzíti a gomb elengedésének pillanatát és a ```release``` változóban tárolja majd kiszámolja, hogy mennyi idő telt el a lenyomás és az elengedés között és a ```delay``` változóba menti. Másrészt, a mért idő alapján az ```encoder``` függvény eldönti, hogy zenét indítson ```start_playing``` vagy leállítson ```stop_playing```. Ha a lenyomás hossza kevesebb mint 1 másodperc akkor az előzőt teszi, ha több akkor az utóbbit. 

Miután elvégeztük azon függvények definiálását, amiket a gomb lenyomásával szeretnénk aktiválni, a gomb megfelelő funkcióihoz hozzá is kell rendelnünk.

In [ ]:
plst.when_pressed = start
plst.when_released = end

next_song.when_pressed = play_next_song
previous_song.when_pressed = play_previous_song

A ```plst``` gombot jelöljük ki, hogy elindítsa és megállítsa a zenék lejátszását, így ezen a gombon kell elvégezni a lenyomás hosszának mérését. Ezért gomb lenyomásához a ```start``` függvényt az elengedéséhez pedig az ```end``` függvényt rendeljük (a függvény nevek után nincs zárójel!!!). 

A másik két gomb esetében nincs szükség idő mérésére, így akár lenyomáskor vagy elengedéskor is elindíthatjuk a hozzárendelt függvényt. Mi a lenyomást választottuk, így a ```when_pressed``` metódust módosítjuk mindkét gombnál.

Végül csak egy végtelen ciklust kell elindítanunk, hogy a python programunk figyelhesse milyen utasításokat küldünk neki a gombok lenyomásával.

In [ ]:
while True:
    pass

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```button_music.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```button_music.py```-t elmentettük. Ott begépelve a ```python button_music.py``` parancsot, letesztelhetjük a programunk működését. Elindítás után a terminál várakozó üzemmódba kerül. Ha megnyomjuk a lejátszás gombot, akkor a terminál kiírja, hogy a zene szerver elindult és zenét kell hallanunk a hangszóróból. Természetesen a zenéinket a program mappájában a ```./Music``` mappában kell tárolnunk, különben a kód nem találja meg őket. Próbáljuk ki az összes beprogramozott gomb funkciót.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Más funkciókat rendelni a gombokhoz, pl. hangerőszabályozás.
* Újabb funkciókat hozzáadni, amit pl. két gomb egyszerre való lenyomásával lehet aktiválni.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) time.time leírása - [https://docs.python.org/3/library/time.html](https://docs.python.org/3/library/time.html)

3) Pygame leírása - [https://www.pygame.org/news](https://www.pygame.org/news)

4) subprocess modul leírás - [https://docs.python.org/3/library/subprocess.html](https://docs.python.org/3/library/subprocess.html)

5) mocp modul leírás - [https://pypi.org/project/mocp/](https://pypi.org/project/mocp/)